In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

!rm -rf ./logs/

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

2021-10-28 00:10:47.566192: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-28 00:10:47.566253: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-6-20211025-160423): /proc/driver/nvidia/version does not exist


[]

In [2]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename):
    data = np.matrix(pd.read_csv(filename).values)

    #we need to transpose data sets to be compatable with our weight vectors
    data_y = data[:, 0]
    data_y-=1 #binaryization of the categorical data

    data_x = data[:, 1:]
    data_x = convert_3D(normalize(data_x))
    return data_x, data_y

In [5]:
training_data_x, training_data_y = features_and_labels("../archive/exoTrain.csv")
test_x, test_y = features_and_labels("../archive/exoTest.csv")

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [ ]:
LSTMmodel = Sequential([
    LSTM(4, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    LSTM(4),
    Dense(1)
])

LSTMmodel.compile(
    loss=BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

LSTMmodel.fit(training_data_x, training_data_y, epochs=3, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback])

model = LSTMmodel
model.save('KerasLSTM/')

2021-10-28 00:10:55.494545: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-28 00:10:56.111773: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 00:10:56.111823: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-28 00:10:56.407185: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-28 00:10:56.517772: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/3
  1/112 [..............................] - ETA: 14:59 - loss: 0.6931 - accuracy: 0.9688

2021-10-28 00:11:04.810143: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-28 00:11:04.810672: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/112 [..............................] - ETA: 7:08 - loss: 0.6910 - accuracy: 0.9844 

2021-10-28 00:11:08.546748: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-28 00:11:09.802917: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-28 00:11:12.225395: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211028-001056/train/plugins/profile/2021_10_28_00_11_09

2021-10-28 00:11:13.603845: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/fit/20211028-001056/train/plugins/profile/2021_10_28_00_11_09/tensorflow-2-6-20211025-160423.trace.json.gz
2021-10-28 00:11:14.395312: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211028-001056/train/plugins/profile/2021_10_28_00_11_09

2021-10-28 00:11:14.395569: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/fit/20211028-001056/train/plu

112/112 [==============================] - 408s 4s/step - loss: 0.4084 - accuracy: 0.9916 - val_loss: 0.1806 - val_accuracy: 0.9941
Epoch 2/3
 42/112 [==========>...................] - ETA: 3:29 - loss: 0.1542 - accuracy: 0.9926

In [ ]:
%tensorboard --logdir logs/fit

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1c58af11-5d4f-40f3-a808-e868f2c28485' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>